In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from koopman.simulation.simulator import rk4_step, simulate
from koopman.simulation.systems import DuffingOscillator
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import FormatStrFormatter
import casadi as ca
from scipy.linalg import expm
from spatialmath.base import *

import jax
import jax.numpy as jnp
from jax import jacfwd, vmap

duffing = DuffingOscillator(DuffingOscillator.Params(delta=0.5))

In [4]:
Jx = jax.jacfwd(duffing.dynamics, argnums=0)
Ju = jax.jacfwd(duffing.dynamics, argnums=1)

In [ ]:
xmin = np.array([-2.0, -2.0])
xmax = np.array([2.0, 2.0])

def normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / np.clip(norms, 1e-8, np.inf)

# Grid resolution
num_points = 20

# Create grid of (theta, omega)
x1_vals = np.linspace(xmin[0], xmax[0], num_points)
x2_vals = np.linspace(xmin[1], xmax[1], num_points)
X1, X2 = np.meshgrid(x1_vals, x2_vals)  # shape (num_points, num_points)

# Flatten grid for vectorized computation
grid_points = np.stack([X1.ravel(), X2.ravel()], axis=-1)  # shape (N^2, 2)
us_null = np.zeros((grid_points.shape[0], duffing.nu))

grid_points_dot = duffing.dynamics(grid_points, us_null)

linearization_xs = np.array([
    [-1.0, 0.0],
    [0.0, 0.0],
    [1.0, 0.0],
])

linearization_us = np.array([
    [0.0],
    [0.0],
    [0.0],
])

As = np.array([ Jx(x0, u0) for x0, u0 in zip(linearization_xs, linearization_us) ])
Bs = np.array([ Ju(x0, u0) for x0, u0 in zip(linearization_xs, linearization_us) ])

def pairwise_distances(A, C):
    # A: (B, nx), C: (M, nx)
    A_exp = A[:, None, :]    # (B, 1, nx)
    C_exp = C[None, :, :]    # (1, M, nx)
    
    diffs = A_exp - C_exp    # (B, M, nx)
    dists = np.linalg.norm(diffs, axis=-1)  # (B, M)
    
    return dists

D = pairwise_distances(grid_points, linearization_xs)

0.11768778828946248
